In [1]:
# 기본 옵션
options(warn = -1)
options(repr.plot.width=15, repr.plot.height=9)
# options(scipen = 100)
Sys.setlocale(category = "LC_ALL", locale = "ko_KR.UTF-8")

[1] "ko_KR.UTF-8/ko_KR.UTF-8/ko_KR.UTF-8/C/ko_KR.UTF-8/C"

# Mushroom
- 8124 종류의 버섯의 특징과 독의 유무를 정리해놓은 데이터셋
- 버섯 모양의 특징을 기반으로 독의 유무를 판단
- 한 줄(ROW)이 버섯 한 종류

1번째 컬럼(독의 유무) : 독(p), 식용(e)<br>
2번째 컬럼(버섯의 머리 모양) : 종 모양(b), 원뿔(c), 볼록(x), 평평(f), 혹(k), 오목(s)<br>
4번째 컬럼(버섯의 머리 색) : 갈색(n), 황갈색(b), 연한갈색(c), 회색(g) 

# 랜덤포레스트를 이용한 버섯 분류 예측

In [17]:
mr <- read.csv("Data/mushroom.csv", header = F, stringsAsFactors = T)

In [18]:
library(randomForest)

In [21]:
# Feature를 숫자화
for(i in 2:ncol(mr)) {
    mr[,i] <- as.numeric(mr[,i])
}

# 정렬(Sampling을 하려면 데이터가 무작위로 있으면 안됨) : V1 기준으로 p와 e를 정렬
library(dplyr)
mr <- arrange(mr,V1)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,p,6,3,5,2,7,2,1,2,5,...,3,8,8,1,3,2,5,3,4,6
2,e,6,3,10,2,1,2,1,1,5,...,3,8,8,1,3,2,5,4,3,2
3,e,1,3,9,2,4,2,1,1,6,...,3,8,8,1,3,2,5,4,3,4
4,p,6,4,9,2,7,2,1,2,6,...,3,8,8,1,3,2,5,3,4,6
5,e,6,3,4,1,6,2,2,1,5,...,3,8,8,1,3,2,1,4,1,2
6,e,6,4,10,2,1,2,1,1,6,...,3,8,8,1,3,2,5,3,3,2


In [37]:
# 정리한 dataset의 식용버섯(e)와 독버섯(p)의 갯수 확인
# 식용버섯의 갯수 : 4208
nrow(subset(mr,V1 == 'e'))
# 독버섯의 갯수 : 3916
nrow(subset(mr,V1 == 'p'))
# 비율(0.93 : 1)
nrow(subset(mr,V1 == 'p')) / nrow(subset(mr,V1 == 'e'))

[1] 4208

[1] 3916

[1] 0.9306084

In [39]:
# Sampling
library(caret)
set.seed(1234)
samp <- createDataPartition(mr$V1, p=0.7, list=F)

In [41]:
# 학습 데이터
data.train <- mr[samp,]
# 검증 데이터
data.test <- mr[-samp,]

In [64]:
# RandomForest로 예측력 확인
# 문제
x <- subset(data.test, select = -V1)
# 정답
y <- data.test$V1
# 학습
model.rf <- randomForest(V1 ~ ., data = data.train, ntree = 100)
# 검증
pred <- predict(model.rf, x)
# 예측력
mean(pred == y)

[1] 1

In [71]:
rfFun <- function(data)  {
    library(randomForest)
    library(caret)
    result <- c()
    for(i in 1:count) {
        samp <- createDataPartition(mr$V1, p=0.7, list=F)
        result <- c(result,
            mean(
                predict(model.rf, subset(data.test, select = -V1)) == data.test$V1)) # 예측력
    }
    cat("\033[45m------------------------------------------------\033[0m\n")
    cat("\033[36m 예측율 \033[0m\n",result,"\n")
    # cat("\n------------------------------------------------\n")
    cat("\033[45m------------------------------------------------\033[0m\n")
    cat("\033[31m 평균 예측율 \033[0m\n",mean(result),"\n")
    # cat("\n------------------------------------------------")
    cat("\033[45m------------------------------------------------\033[0m\n")
}

In [73]:
rfFun(10)

------------------------------------------------
 예측율 
 1 1 1 1 1 1 1 1 1 1 
------------------------------------------------
 평균 예측율 
 1 
------------------------------------------------


In [100]:
# 인공신경망을 이용한 예측력 확인
# library(nnet)
model.nnet <- nnet(V1 ~ ., data = data.train, size = 3)
# predict(model.nnet, x, type = "class")
cat("인공신경망 : ", mean(predict(model.nnet, x, type = "class") == y),"\n")

# SVM을 이용한 예측력 확인
library(e1071)
model.svm <- svm(V1 ~ ., data=data.train)
cat("SVM :",mean(predict(model.svm, x) == y),"\n")

# 로지스틱 회귀분석을 이용한 예측력 확인
model.multinom <- multinom(V1 ~ ., data = data.train)
cat("로지스틱 회귀분석 :",mean(predict(model.multinom, x) == y),"\n")

# weights:  73


initial  value 4564.474242 
iter  10 value 3668.842301
iter  20 value 2052.308629
iter  30 value 1600.675419
iter  40 value 1151.023576
iter  50 value 1120.542323
iter  60 value 1101.407950
iter  70 value 1100.087299
iter  80 value 1100.053644
iter  90 value 1100.043506
iter 100 value 1100.038108
final  value 1100.038108 
stopped after 100 iterations
인공신경망 :  0.9449918 
SVM : 1 
# weights:  24 (23 variable)
initial  value 3942.621163 
iter  10 value 1633.673376
iter  20 value 1264.137543
iter  30 value 868.476686
iter  40 value 789.424077
iter  50 value 785.517204
final  value 785.517135 
converged
로지스틱 회귀분석 : 0.9671593 


---
# One-Hot Encoding
- Feature Engineering의 한 분야

In [108]:
# 각 컬럼별 Data 확인
for(i in 2:ncol(mr)) {
    cat("Column Name : ", colnames(mr)[i],"\n")
}

Column Name :  V2 
Column Name :  V3 
Column Name :  V4 
Column Name :  V5 
Column Name :  V6 
Column Name :  V7 
Column Name :  V8 
Column Name :  V9 
Column Name :  V10 
Column Name :  V11 
Column Name :  V12 
Column Name :  V13 
Column Name :  V14 
Column Name :  V15 
Column Name :  V16 
Column Name :  V17 
Column Name :  V18 
Column Name :  V19 
Column Name :  V20 
Column Name :  V21 
Column Name :  V22 
Column Name :  V23 


In [110]:
# 작업 후의 컬럼 갯수 파악
num <- c()
for(i in 2:23) {
    num = append(num, length(unique(mr[,i])))
}
num
sum(num)

[1]  6  4 10  2  9  2  2  2 12  2  5  4  4  9  9  1  4  3  5  9  6  7

[1] 117

## 별도의 DataFrame 생성

In [111]:
mr2 <- data.frame(V1=mr$V1)

In [115]:
# col의 종류 확인
sort(unique(mr$V2))

[1] 1 2 3 4 5 6

In [ ]:
for(i in 2:23) {
    for(j in sort(unique(mr[,i]))) {
        mr2 <- cbind(mr2, mr[,i] == j)
    }
}

colnames(mr2) <- paste("V",1:118, sep = "")

---
## Column 이름과 One-Hot Encoding을 한번에 처리

In [120]:
mr4 <- data.frame(V1 = mr$V1)

In [121]:
count = 2 # 2번째 컬럼부터 작업 시작 후 원본 컬럼번호에 새로운 컬럼이름 저장하기 위해 사용
for(i in 2:23) {
    for(j in sort(unique(mr[,i]))) {
        mr4 <- cbind(mr4, mr[,i] == j)
        colnames(mr4)[count] <- paste("V",i,"_",j,sep = "")
        count <- count + 1
    }
}

In [131]:
createDataPartition(mr4$V1 ~ ., p=0.7, list = F)

ERROR: Error in unique.default(x, nmax = nmax): unique()는 오로지 벡터들에만 적용됩니다


ERROR: Error in unique.default(mr4$V1 ~ .): unique()는 오로지 벡터들에만 적용됩니다
